In [1]:
import tensorflow as tf
# import tensorflow.contrib.eager as tfe
# tfe.enable_eager_execution()
import sys
sys.path.append(r'C:/Users/ua12/Desktop/Pose Estimation Keras/')
import cnn
from cnn import conv_base
from build_dataset import Dataset
import numpy as np

from keras import layers
from keras import models
from keras import optimizers
import keras.backend as K
from keras.callbacks import TensorBoard, ModelCheckpoint 

from time import time
import cv2
import os 


Using TensorFlow backend.
C:\Users\ua12\AppData\Local\conda\conda\envs\DLCdependencies\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [2]:
# Some Hyperparameters
total_frames = 5
frame_height = 224
frame_width = 224
frame_channels = 3
num_bodyparts = 6

In [3]:
def build_deconv_model():
    frame_tensor = conv_base.layers[0].get_input_at(0)
    resnet_output = cnn.convolution_network(frame_tensor)
    model_resnet = models.Model(inputs=[frame_tensor], outputs=[resnet_output])
    
    deconv_output = cnn.deconvolution_network(model_resnet.output)
    model_deconv = models.Model(inputs=[frame_tensor], outputs=[deconv_output])
    
    losses = {
        "deconvolution_network_output": "binary_crossentropy",
#         "spatial_fusion_output": "binary_crossentropy",
#         "optical_flow_output": "binary_crossentropy"
    }
    lossWeights = {"deconvolution_network_output": 1.0,}
#                    "spatial_fusion_output": 1.0,
#                    "optical_flow_output": 1.0}
    
    
    opt = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
    model_deconv.compile( optimizer=opt, loss=losses, loss_weights=lossWeights, metrics = ["accuracy"])
    
    return model_deconv

model = build_deconv_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 14, 14, 6)         110598    
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 28, 28, 6)         330       
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 56, 56, 6)         330       
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 112, 112, 6)       330       
_________________________________________________________________
deconvolution_network_output (None, 224, 224, 6)       330       
Total para

In [4]:
import csv
def parse_csv(file):
    with open(file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        indices = []
        for line in csv_reader:
            indices.extend([int(f) for f in line if f!=''])
    return indices

indices = parse_csv('dataset_indices.csv')
print(indices)

[0, 2, 3, 4, 6, 7, 8, 11, 15, 16, 17, 18, 19, 20, 23, 120, 126, 127, 178, 185, 199, 228, 244, 261, 267, 285, 306, 335, 367, 375, 389, 405, 410, 415, 420, 426, 430, 436, 440, 445, 452, 455, 460, 465, 470, 476, 480, 485, 490, 496, 500, 511, 520, 525, 530, 535, 542, 552, 555, 568, 570, 575, 581, 588, 595, 600, 605, 610, 615, 621, 625, 633, 636, 640, 645, 650, 655, 665, 670, 675, 680, 685, 690, 695, 700, 705, 710, 715, 718, 723, 725, 751, 755, 770, 783, 784, 804, 812, 816, 823, 829, 838, 841, 845, 850, 855, 863, 865, 891, 894, 904, 908, 915, 920, 929, 946, 953, 955, 959, 965, 970, 980, 987, 990, 995, 1030, 1035, 1041, 1045, 1160, 1164, 1170, 1174, 1220, 1225, 1230, 1235, 1240, 1245, 1250, 1255, 1267, 1275, 1280, 1285, 1290]


In [5]:
import h5py
with h5py.File('dataset.h5', 'r') as hf:
    central_heatmap = hf['central_heatmap'][indices]
    frames = hf['frames'][indices]
    heatmaps_individual = hf['heatmaps_individual'][indices]
    heatmaps_combined = hf['heatmaps_combined'][indices]

In [6]:
home_dir = os.getcwd()
central_heatmap = np.expand_dims(central_heatmap, axis=-1)
heatmaps_combined = np.expand_dims(heatmaps_combined, axis=-1)
heatmaps_individual = np.swapaxes(heatmaps_individual, 2, 4)
print(central_heatmap.shape)
print(heatmaps_individual.shape)
print(heatmaps_combined.shape)

(146, 6, 224, 224, 1)
(146, 5, 224, 224, 6)
(146, 5, 224, 224, 1)


In [7]:
checkpointer = ModelCheckpoint(filepath= os.path.join(home_dir, "pretrained_models/model010919.h5"),
                               verbose=1,
                               monitor='loss')

tensorboard = TensorBoard(log_dir=os.path.join(home_dir, "logs/{}".format(time())))


In [8]:
def generator(frames, heatmaps):
    while(1):
        for j,l in zip(frames, heatmaps):
            # include batch dimension and repeat the matrix at axis 0 num_frames times to match the target dimensionality
            yield (j, {
#                 "optical_flow_output" : np.repeat(np.expand_dims(k, axis=0), 5, 0), 
                       "deconvolution_network_output": l}) 
            
model.fit_generator(generator(frames,
                                     heatmaps_individual),
                           steps_per_epoch=386, 
                           verbose = 2,
                           epochs = 120,
                           callbacks=[ checkpointer, tensorboard])

Epoch 1/120


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node resnet50/conv1/convolution}} = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 2, 2], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](resnet50/conv1/convolution-0-TransposeNHWCToNCHW-LayoutOptimizer, conv1/kernel/read)]]
	 [[{{node metrics/acc/Mean_1/_2875}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_3337_metrics/acc/Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]